In [ ]:
from google.colab import drive
drive.mount('/content/drive')


Mounted at /content/drive


In [ ]:
!pip install split_folders

In [ ]:

!pip install keras_applications

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.7/50.7 kB 2.3 MB/s eta 0:00:00


In [ ]:
import json
import os
from distutils.dir_util import copy_tree
import shutil
import pandas as pd

# TensorFlow and tf.keras
import tensorflow as tf
from tensorflow.keras import backend as K
print('TensorFlow version: ', tf.__version__)

dataset_path = '/content/drive/MyDrive/deepfake project final/DeepFake-Detect/split_dataset'

tmp_debug_path = './tmp_debug'
print('Creating Directory: ' + tmp_debug_path)
os.makedirs(tmp_debug_path, exist_ok=True)

def get_filename_only(file_path):
    file_basename = os.path.basename(file_path)
    filename_only = file_basename.split('.')[0]
    return filename_only

from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications import EfficientNetB0
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint
from tensorflow.keras.models import load_model

input_size = 128
batch_size_num = 16
train_path = os.path.join(dataset_path, 'train')
val_path = os.path.join(dataset_path, 'val')
test_path = os.path.join(dataset_path, 'test')

train_datagen = ImageDataGenerator(
    rescale = 1/255,    # Rescale the tensor values to [0,1]
    rotation_range = 10,
    width_shift_range = 0.1,
    height_shift_range = 0.1,
    shear_range = 0.2,
    zoom_range = 0.1,
    horizontal_flip = True,
    fill_mode = 'nearest'
)

train_generator = train_datagen.flow_from_directory(
    directory=train_path,
    target_size=(input_size, input_size),
    batch_size=batch_size_num,
    class_mode='binary',  # Binary classification
    shuffle=True
)

val_datagen = ImageDataGenerator(
    rescale = 1/255    # Rescale the tensor values to [0,1]
)

val_generator = val_datagen.flow_from_directory(
    directory=val_path,
    target_size=(input_size, input_size),
    batch_size=batch_size_num,
    class_mode='binary',  # Binary classification
    shuffle=False  # Disable shuffling
)

test_datagen = ImageDataGenerator(
    rescale = 1/255    # Rescale the tensor values to [0,1]
)

test_generator = test_datagen.flow_from_directory(
    directory=test_path,
    classes=['real', 'fake'],
    target_size=(input_size, input_size),
    color_mode="rgb",
    class_mode=None,
    batch_size=1,
    shuffle=False
)

# Train a CNN classifier
efficient_net = EfficientNetB0(
    weights='imagenet',
    input_shape=(input_size, input_size, 3),
    include_top=False,
    pooling='max'
)

model = Sequential()
model.add(efficient_net)
model.add(Dense(units=512, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(units=128, activation='relu'))
model.add(Dense(units=1, activation='sigmoid'))

# Compile model
model.compile(optimizer=Adam(learning_rate=0.0001), loss='binary_crossentropy', metrics=['accuracy'])

checkpoint_filepath = './tmp_checkpoint'
print('Creating Directory: ' + checkpoint_filepath)
os.makedirs(checkpoint_filepath, exist_ok=True)

custom_callbacks = [
    EarlyStopping(
        monitor='val_loss',
        mode='min',
        patience=5,
        verbose=1
    ),
    ModelCheckpoint(
        filepath=os.path.join(checkpoint_filepath, 'best_model.keras'),  # Save as .keras file
        monitor='val_loss',
        mode='min',
        verbose=1,
        save_best_only=True
    )
]

# Train network
num_epochs = 8
history = model.fit(
    train_generator,
    epochs=num_epochs,
    steps_per_epoch=len(train_generator),  # Ensure steps_per_epoch is correctly defined
    validation_data=val_generator,
    validation_steps=len(val_generator),  # Ensure validation_steps is set to match the size of val_generator
    callbacks=custom_callbacks,
    verbose=2  # Verbose logging to track the training process
)

print(history.history)

# Load the saved model that is considered the best
best_model = load_model(os.path.join(checkpoint_filepath, 'best_model.keras'))  # Ensure consistent loading

# Generate predictions
test_generator.reset()

preds = best_model.predict(
    test_generator,
    verbose=1
)

# Flatten predictions for easier handling
test_results = pd.DataFrame({
    "Filename": test_generator.filenames,
    "Prediction": preds.flatten()
})

print(test_results)


TensorFlow version:  2.17.0
Creating Directory: ./tmp_debug
Found 16 images belonging to 2 classes.
Found 2 images belonging to 2 classes.
Found 4 images belonging to 2 classes.
Creating Directory: ./tmp_checkpoint
Epoch 1/8



Epoch 1: val_loss improved from inf to 0.70354, saving model to ./tmp_checkpoint/best_model.keras
1/1 - 59s - 59s/step - accuracy: 0.3750 - loss: 1.9758 - val_accuracy: 0.5000 - val_loss: 0.7035
Epoch 2/8


AttributeError: 'NoneType' object has no attribute 'items'